In [1]:
import scrapy
import json
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
import xmltodict
import pymongo
class MegatimingSpider(scrapy.Spider):
    name = "MegatimingSpider"
    errors = [];
    def start_requests(self):
        urls = [];
        
        for page in range(1,63):
            urls.append('http://megatiming.pl/contests/regional/'+str(page));
       
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
    
    def parse(self, response):
        js_script = response.xpath('/html/body/script[contains(., "contests")]/text()').get()
        json_source=js_script[js_script.find('data=')+6:-1]       
        json_source_1 =  json_source[json_source.find('"contests"')+11:] 
        json_source_2 = json_source_1[:json_source_1.find('"home"')-1];
        data = json.loads(json_source_2)
        for key in data["data"]["contests"].keys():
            #print(key)
            yield scrapy.Request(url="http://megatiming.pl/contest/"+key,
                             callback=self.parseContest,
                             cb_kwargs=dict(uuid=key))
        
    def parseContest(self,response,uuid):
        fileName = self.getFileName(response,uuid)
        if (fileName):
            self.writeToDatabase(fileName,uuid)
    
    def getFileName(self,response,uuid):
        try:
            js_script = response.xpath('/html/body/script[contains(., "contests")]/text()').get()
            json_source_1=js_script[js_script.find('"contests"')+11:-1]
            json_source_2=json_source_1[:json_source_1.find('"home"')-1]
            data = json.loads(json_source_2)
            for element in data["data"]["contests"][uuid]["files"]["result"]:
                if (element["url"].find("lxf")>=0):
                    return(element["url"])
            return None
        except:
            return None
    
    def writeToDatabase(self,url,uuid):
        try:
            resp = urlopen(url)
            zipfile = ZipFile(BytesIO(resp.read()))
            nameFileLef = zipfile.namelist()[0]
            xmlFile = zipfile.read(nameFileLef).decode("utf-8") 
        
            # change wrong name attributes
            #result.url, organizer.url, hostclub.url -> result_url , organizer_url , hostclub_url
            xmlFile = xmlFile.replace("result.url","result_url");
            xmlFile = xmlFile.replace("organizer.url","organizer_url");
            xmlFile = xmlFile.replace("hostclub.url","hostclub_url");
            xmlFile = xmlFile.replace("shortname.en","shortname_en")
            xmlFile = xmlFile.replace("firstname.en","firstname_en")
            xmlFile = xmlFile.replace("name.en","name_en")
            xmlFile = xmlFile.replace("lastname.en","lastname_en")
            xmlFile = xmlFile.replace("city.en","city_en")
        
            # read to dictionary
            data = xmltodict.parse(xmlFile)
            data["uuid"] = uuid
            # open mongodb and write dictionary
            myclient = pymongo.MongoClient("mongodb://localhost:27017/")
            mydb = myclient["megatiming"]
            mycol = mydb["results"]
            mycol.insert_one(data)
       
        except Exception as e:
            self.errors.append(uuid+" error "+str(e))
    

In [2]:
from scrapy.crawler import CrawlerProcess
process = CrawlerProcess()
spider = MegatimingSpider
process.crawl(spider)
process.start()

2020-01-08 15:28:52 [scrapy.utils.log] INFO: Scrapy 1.8.0 started (bot: scrapybot)
2020-01-08 15:28:52 [scrapy.utils.log] INFO: Versions: lxml 4.4.2.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.6.9 (default, Nov  7 2019, 10:44:02) - [GCC 8.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Linux-5.0.0-37-generic-x86_64-with-LinuxMint-19.3-tricia
2020-01-08 15:28:52 [scrapy.crawler] INFO: Overridden settings: {}
2020-01-08 15:28:52 [scrapy.extensions.telnet] INFO: Telnet Password: 80f4d1adeee0fc1d
2020-01-08 15:28:52 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2020-01-08 15:28:52 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.Down

2020-01-08 15:29:02 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/1dd73c49-f97c-4096-bb85-684b0eb2dbb0> (referer: http://megatiming.pl/contests/regional/12)
2020-01-08 15:29:02 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/8e0921ac-65f8-45ca-ba1b-52ebc8afc3da> (referer: http://megatiming.pl/contests/regional/22)
2020-01-08 15:29:02 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/0f026e83-c8bd-40aa-9a16-d2df8e10de3c> (referer: http://megatiming.pl/contests/regional/22)
2020-01-08 15:29:02 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/7612b158-7c5c-4622-ae7f-dea0aeffd1d1> (referer: http://megatiming.pl/contests/regional/22)
2020-01-08 15:29:03 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/8a482da5-0344-4204-b270-cfb5e3b827d3> (referer: http://megatiming.pl/contests/regional/22)
2020-01-08 15:29:03 [scrapy.core.engine] DEBUG: Crawled (200) <GE

2020-01-08 15:29:32 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/917b9d65-fc44-4a08-80e8-1417425a779d> (referer: http://megatiming.pl/contests/regional/25)
2020-01-08 15:29:32 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/a07f0b3c-a9f1-4a23-8955-a87e8ec51a17> (referer: http://megatiming.pl/contests/regional/25)
2020-01-08 15:29:32 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/cbc061ed-5cf2-4913-9199-9ab6e94b13ff> (referer: http://megatiming.pl/contests/regional/25)
2020-01-08 15:29:38 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/b72000fe-bc2c-4b58-83e5-67118588b1b2> (referer: http://megatiming.pl/contests/regional/25)
2020-01-08 15:29:38 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/8ec7bc45-53c3-4ada-b322-53d82bc3b769> (referer: http://megatiming.pl/contests/regional/25)
2020-01-08 15:29:38 [scrapy.core.engine] DEBUG: Crawled (200) <GE

2020-01-08 15:29:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/2896e195-e2b6-4217-8da1-ad4a0caa9300> (referer: http://megatiming.pl/contests/regional/23)
2020-01-08 15:29:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/d840f705-91ed-40b0-b815-9bd41c861392> (referer: http://megatiming.pl/contests/regional/23)
2020-01-08 15:29:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/dc2a0bc6-fc63-4784-810c-8b47efaa48ca> (referer: http://megatiming.pl/contests/regional/23)
2020-01-08 15:29:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/4f6a17b0-2021-4f96-8c38-24448335cc57> (referer: http://megatiming.pl/contests/regional/23)
2020-01-08 15:29:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/bc2ea814-a14e-4cbe-bc98-88ab5cf9f4c2> (referer: http://megatiming.pl/contests/regional/23)
2020-01-08 15:29:57 [scrapy.core.engine] DEBUG: Crawled (200) <GE

2020-01-08 15:30:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/da62e168-d223-4318-93a4-c275c30faa31> (referer: http://megatiming.pl/contests/regional/24)
2020-01-08 15:30:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/b5e46272-34b9-486f-badd-0b86d422eaef> (referer: http://megatiming.pl/contests/regional/24)
2020-01-08 15:30:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/bf908ce1-ec95-4a08-9f1a-552e066ce950> (referer: http://megatiming.pl/contests/regional/24)
2020-01-08 15:30:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/292eff64-554a-4ff4-8301-c339966282f4> (referer: http://megatiming.pl/contests/regional/24)
2020-01-08 15:30:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/c69e894a-d974-43c1-b1d0-32aba5f5c207> (referer: http://megatiming.pl/contests/regional/24)
2020-01-08 15:30:08 [scrapy.core.engine] DEBUG: Crawled (200) <GE

...

2020-01-08 15:44:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/34ac6800-ea2d-403b-8d54-dd50c0dd53fc> (referer: http://megatiming.pl/contests/regional/44)
2020-01-08 15:44:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/160d986a-817a-4d95-96e6-a83ecabc6a48> (referer: http://megatiming.pl/contests/regional/43)
2020-01-08 15:44:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/924f4f24-7626-4d12-9c7a-75cc881a0eed> (referer: http://megatiming.pl/contests/regional/43)
2020-01-08 15:44:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/aaae282d-1fe0-4672-bed1-a913e29cf523> (referer: http://megatiming.pl/contests/regional/44)
2020-01-08 15:44:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/2c2cd74b-7ede-475d-a91d-ca358231e424> (referer: http://megatiming.pl/contests/regional/43)
2020-01-08 15:44:30 [scrapy.core.engine] DEBUG: Crawled (200) <GE

2020-01-08 15:44:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/0c96ea5a-0cdf-41b3-a17e-ec4248df0ee8> (referer: http://megatiming.pl/contests/regional/48)
2020-01-08 15:44:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/e1250809-18bc-4234-89b3-11f72a4a940a> (referer: http://megatiming.pl/contests/regional/48)
2020-01-08 15:44:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/8e20237e-394f-46a3-a7c3-41630d72fd31> (referer: http://megatiming.pl/contests/regional/48)
2020-01-08 15:44:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/fd8d1b28-4e80-4183-b1d1-4e33be8395d4> (referer: http://megatiming.pl/contests/regional/48)
2020-01-08 15:44:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/e4e2a906-b748-42da-a428-f46a44624aa2> (referer: http://megatiming.pl/contests/regional/48)
2020-01-08 15:44:34 [scrapy.core.engine] DEBUG: Crawled (200) <GE

2020-01-08 15:44:39 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/0d1b9575-5859-42c5-9aa8-3b10a39ee515> (referer: http://megatiming.pl/contests/regional/47)
2020-01-08 15:44:39 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/a686da18-e782-4722-ae73-44501f3fdb80> (referer: http://megatiming.pl/contests/regional/47)
2020-01-08 15:44:39 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/0faa65ba-dff3-450b-b81b-2c35cb37582a> (referer: http://megatiming.pl/contests/regional/47)
2020-01-08 15:44:40 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/cd917515-b2a1-44e3-abb5-cd231282d8f5> (referer: http://megatiming.pl/contests/regional/47)
2020-01-08 15:44:40 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/c204bc67-a96e-41dd-b60e-c5ec0978560c> (referer: http://megatiming.pl/contests/regional/47)
2020-01-08 15:44:40 [scrapy.core.engine] DEBUG: Crawled (200) <GE

2020-01-08 15:44:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/1fcea297-d587-41f0-8e9d-8e82eeae38bc> (referer: http://megatiming.pl/contests/regional/46)
2020-01-08 15:44:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/f828e2a3-4086-42e6-a678-a74577bbf0d6> (referer: http://megatiming.pl/contests/regional/46)
2020-01-08 15:44:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/c97abb3f-2f9c-415b-8c0a-469f1b7210d2> (referer: http://megatiming.pl/contests/regional/46)
2020-01-08 15:44:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/c86e2f07-88a3-46e0-9201-028ad0d82ef2> (referer: http://megatiming.pl/contests/regional/46)
2020-01-08 15:44:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/14378b50-f11f-440f-b7bf-ca1b21329062> (referer: http://megatiming.pl/contests/regional/47)
2020-01-08 15:44:43 [scrapy.core.engine] DEBUG: Crawled (200) <GE

2020-01-08 15:44:48 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/c440626c-fbcf-4c6b-a1f0-65d4e6bf1030> (referer: http://megatiming.pl/contests/regional/46)
2020-01-08 15:44:48 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/7a7f6ff4-1259-48b1-8880-6f8f5109a233> (referer: http://megatiming.pl/contests/regional/46)
2020-01-08 15:44:48 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/854f5072-637e-4b3a-884c-faae91855898> (referer: http://megatiming.pl/contests/regional/46)
2020-01-08 15:44:48 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/67b08bd2-5f49-4ab9-9df8-0f8d234fb017> (referer: http://megatiming.pl/contests/regional/46)
2020-01-08 15:44:48 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://megatiming.pl/contest/21774db8-20ea-4169-ba1a-5cf35eb99e60> (referer: http://megatiming.pl/contests/regional/46)
2020-01-08 15:44:48 [scrapy.core.engine] DEBUG: Crawled (200) <GE

In [3]:
spider.errors

['5aad8d59-b57e-4040-b7ee-a3ddc4ec6cf8 error not well-formed (invalid token): line 3554, column 64',
 'ce70bddc-de7f-4e01-9854-42c261924dd9 error not well-formed (invalid token): line 2666, column 64',
 '2548828d-efc0-49e6-a9a1-2ef818324101 error not well-formed (invalid token): line 1834, column 43',
 '3e8d3eb2-a4c2-45b2-91b5-466ee2f8ecac error not well-formed (invalid token): line 1427, column 43',
 "5b49b284-20ea-4687-a6cf-42351c580cb4 error 'utf-8' codec can't decode byte 0xf1 in position 14606: invalid continuation byte",
 "adae05da-5ba7-4673-abc4-9f462efe06f1 error 'utf-8' codec can't decode byte 0xb3 in position 95: invalid start byte",
 "77d3e45d-3053-4d7d-bfbb-34242329fee4 error 'utf-8' codec can't decode byte 0xf1 in position 441: invalid continuation byte",
 "55389bbe-292e-465a-92ad-48bb2c41b54f error 'utf-8' codec can't decode byte 0xb3 in position 95: invalid start byte",
 "5bbd8205-c9cb-4b0a-8416-31d523616c2e error 'utf-8' codec can't decode byte 0xb3 in position 95: inva